In [12]:
import numpy as np
import pandas as pd

def fetch_data(x=0):
    gen = pd.read_csv('fma_metadata/genres.csv')
    fet = pd.read_csv('fma_metadata/features.csv', dtype=object, nrows=5)
    return fet.iloc[:x] if x else fet

def get_song_features(fn):
    pass

def main():
    data = fetch_data()
    print(data)
    
    
main()

      feature       chroma_cens     chroma_cens.1     chroma_cens.2  \
0  statistics          kurtosis          kurtosis          kurtosis   
1      number                01                02                03   
2    track_id               NaN               NaN               NaN   
3           2  7.1806526184e+00  5.2303090096e+00  2.4932080507e-01   
4           3  1.8889633417e+00  7.6053929329e-01  3.4529656172e-01   

      chroma_cens.3     chroma_cens.4     chroma_cens.5     chroma_cens.6  \
0          kurtosis          kurtosis          kurtosis          kurtosis   
1                04                05                06                07   
2               NaN               NaN               NaN               NaN   
3  1.3476201296e+00  1.4824777842e+00  5.3137123585e-01  1.4815930128e+00   
4  2.2952005863e+00  1.6540306807e+00  6.7592434585e-02  1.3668476343e+00   

      chroma_cens.7     chroma_cens.8  ...        tonnetz.39  \
0          kurtosis          kurtosis  ...    